In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit


In [2]:
data = pd.read_csv("../data/HMIS_DATA_CORRECTED_17_21/mh_dist17_21_with_IDs_date_correction.csv")
data = data[(data['indicator_type'] == 'Total [(A+B) or (C+D)]')]
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data.index = pd.DatetimeIndex(data.index)

In [4]:
def huber_regression_lags_only(
    series,
    district_name,
    max_lags=3,
    differencing=True,
    epsilon=1.35,
    alpha=0.0001,
    max_iter=1000,
    use_cv=False,
    cv_params=None,
    random_state=None
):
    """
    Robust Huber regression for time series forecasting with:
    - Automatic hyperparameter tuning via grid search
    - Outlier-resistant modeling
    - Enhanced differencing checks
    """
    # Create directory structure
    os.makedirs('HuberRegression', exist_ok=True)
    
    # 1. Stationarity and Differencing (fixed adf_result check)
    original_series = series.copy()
    d = 0
    if differencing:
        adf_result = adfuller(series.dropna())
        if adf_result[1] > 0.05:  # Fixed p-value access
            d = 1
            series = series.diff().dropna()

    # 2. Feature Engineering - Lag features with type consistency
    df = pd.DataFrame({'y': series.astype(np.float64)})
    for lag in range(1, max_lags + 1):
        df[f'lag_{lag}'] = df['y'].shift(lag).astype(np.float64)
    df = df.dropna()

    # 3. Time-based split with validation
    train_size = int(len(df) * 0.8)
    train = df.iloc[:train_size]
    test = df.iloc[train_size:]

    X_train = train.drop(columns=['y'])
    y_train = train['y']
    X_test = test.drop(columns=['y'])
    y_test = test['y']

    # 4. Model training with optional CV
    if use_cv:
        # Default parameter grid
        param_grid = cv_params or {
            'epsilon': [1.0, 1.35, 1.5, 2.0],
            'alpha': [0.0001, 0.001, 0.01, 0.1]
        }
        
        grid_search = GridSearchCV(
            HuberRegressor(max_iter=max_iter),
            param_grid,
            cv=5,
            scoring='neg_mean_squared_error'
        )
        grid_search.fit(X_train, y_train)
        model = grid_search.best_estimator_
        best_epsilon = model.epsilon
        best_alpha = model.alpha
    else:
        model = HuberRegressor(
            epsilon=epsilon,
            alpha=alpha,
            max_iter=max_iter
        )
        model.fit(X_train, y_train)
        best_epsilon = epsilon
        best_alpha = alpha

    # 5. Forecasting with type consistency
    pred_test = model.predict(X_test).astype(np.float64)

    # 6. Inverse differencing with edge case handling
    if d == 1:
        last_train_value = original_series.iloc[len(original_series) - len(test) - 1]
        pred_test = np.cumsum(pred_test) + last_train_value
        y_test = original_series.iloc[-len(test):]

    # 7. Enhanced DataFrame with confidence intervals
    forecast_df = pd.DataFrame({
        'district': district_name,
        'date': y_test.index,
        'actual': y_test.values,
        'forecast': pred_test,
        'residual': y_test.values - pred_test
    })

    # 8. Robust CSV saving
    forecast_csv_path = os.path.join('HuberRegression', 'huber_forecasts.csv')
    forecast_df.to_csv(
        forecast_csv_path,
        mode='a',
        header=not os.path.exists(forecast_csv_path),
        index=False
    )
     # 9. Comprehensive metrics
    rmse = np.sqrt(mean_squared_error(forecast_df['actual'], forecast_df['forecast']))
    metrics_data = {
        'district': district_name,
        'rmse': rmse,
        'r_squared': model.score(X_train, y_train),
        'coefficients': model.coef_.tolist(),
        'intercept': model.intercept_,
        'final_epsilon': best_epsilon,
        'final_alpha': best_alpha,
        'differencing': d,
        'n_iterations': model.n_iter_
    }

    if use_cv:
        metrics_data.update({
            'best_params': grid_search.best_params_,
            'cv_results': grid_search.cv_results_
        })

    metrics_df = pd.DataFrame([metrics_data])
    
    metrics_csv_path = os.path.join('HuberRegression', 'huber_metrics.csv')
    metrics_df.to_csv(
        metrics_csv_path,
        mode='a',
        header=not os.path.exists(metrics_csv_path),
        index=False
    )

    # 10. Enhanced plotting with residuals
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
    
    # Forecast plot
    ax1.plot(original_series.index, original_series.values, label='Original Series')
    ax1.plot(forecast_df['date'], forecast_df['forecast'], 
            label='Huber Forecast', color='darkorange')
    ax1.set_title(f'Huber Regression Forecast for {district_name}\nRMSE: {rmse:.2f}')
    ax1.legend()

    # Residual plot
    ax2.bar(forecast_df['date'], forecast_df['residual'], 
           color='gray', alpha=0.7, width=5)
    ax2.axhline(0, color='red', linestyle='--')
    ax2.set_title('Forecast Residuals')
    
    plt.tight_layout()
    plt.savefig(os.path.join('HuberRegression', f'huber_forecast_{district_name}.png'))
    plt.close()

    return {
        'district': district_name,
        'forecast_df': forecast_df,
        'metrics_df': metrics_df,
        'model_params': model.get_params(),
        'feature_names': X_train.columns.tolist()
    }

In [ ]:
districts = data['district'].unique()
rmse_values = []

def run_for_each_district():
    results = {}
    
    for district in districts:
        district_data = data[data['district'] == district]
        ts = district_data["I1"].asfreq('MS')
        
        results = huber_regression_lags_only(
            ts,
            district,
            use_cv=True,
            cv_params={
                'epsilon': [1.0, 1.35, 1.7],
                'alpha': [0.0001, 0.001, 0.01]
            },
            max_iter=2000
        )
        
    
        # Show results
        print("=== Metrics ===")
        print(results['metrics_df'])
        print("\n=== Forecast Data ===")
        print(results['forecast_df'].head())
    
    return results
run_for_each_district()

=== Metrics ===
     district        rmse  r_squared  \
0  AHMEDNAGAR  588.015896  -0.464927   

                                        coefficients  intercept  \
0  [0.4700137099527392, 0.44304296172132407, 0.08...   0.000268   

   final_epsilon  final_alpha  differencing  n_iterations  \
0           1.35        0.001             0            26   

                         best_params  \
0  {'alpha': 0.001, 'epsilon': 1.35}   

                                          cv_results  
0  {'mean_fit_time': [0.011429071426391602, 0.006...  

=== Forecast Data ===
     district       date  actual     forecast    residual
0  AHMEDNAGAR 2020-07-01  6478.0  7212.238050 -734.238050
1  AHMEDNAGAR 2020-08-01  5975.0  6734.255051 -759.255051
2  AHMEDNAGAR 2020-09-01  6550.0  6264.023135  285.976865
3  AHMEDNAGAR 2020-10-01  6471.0  6277.611284  193.388716
4  AHMEDNAGAR 2020-11-01  6107.0  6452.380988 -345.380988
=== Metrics ===
  district        rmse  r_squared  \
0    AKOLA  186.537929  -0.427

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nauti\AppData\

=== Metrics ===
     district       rmse  r_squared  \
0  SINDHUDURG  94.912788   0.028841   

                                        coefficients   intercept  \
0  [0.15704254205526638, -0.03398012516033517, 0....  536.717076   

   final_epsilon  final_alpha  differencing  n_iterations  \
0            1.7         0.01             0            66   

                       best_params  \
0  {'alpha': 0.01, 'epsilon': 1.7}   

                                          cv_results  
0  {'mean_fit_time': [0.007752418518066406, 0.005...  

=== Forecast Data ===
     district       date  actual    forecast    residual
0  SINDHUDURG 2020-07-01   721.0  705.283767   15.716233
1  SINDHUDURG 2020-08-01   549.0  690.501550 -141.501550
2  SINDHUDURG 2020-09-01   579.0  665.648222  -86.648222
3  SINDHUDURG 2020-10-01   530.0  670.178541 -140.178541
4  SINDHUDURG 2020-11-01   476.0  646.866973 -170.866973
=== Metrics ===
  district        rmse  r_squared  \
0  SOLAPUR  561.800205    0.05896   

  

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nauti\AppData\

=== Metrics ===
          district        rmse  r_squared  \
0  MUMBAI SUBURBAN  853.134504   0.876567   

                                        coefficients  intercept  \
0  [1.0702981921994807, -0.31220241814288197, 0.3...   0.000681   

   final_epsilon  final_alpha  differencing  n_iterations  \
0            1.7       0.0001             0            37   

                         best_params  \
0  {'alpha': 0.0001, 'epsilon': 1.7}   

                                          cv_results  
0  {'mean_fit_time': [0.01767449378967285, 0.0114...  

=== Forecast Data ===
          district       date   actual      forecast     residual
0  MUMBAI SUBURBAN 2021-02-01  13444.0  13935.123643  -491.123643
1  MUMBAI SUBURBAN 2021-03-01  13294.0  14396.032002 -1102.032002


{'district': 'MUMBAI SUBURBAN',
 'forecast_df':           district       date   actual      forecast     residual
 0  MUMBAI SUBURBAN 2021-02-01  13444.0  13935.123643  -491.123643
 1  MUMBAI SUBURBAN 2021-03-01  13294.0  14396.032002 -1102.032002,
 'metrics_df':           district        rmse  r_squared  \
 0  MUMBAI SUBURBAN  853.134504   0.876567   
 
                                         coefficients  intercept  \
 0  [1.0702981921994807, -0.31220241814288197, 0.3...   0.000681   
 
    final_epsilon  final_alpha  differencing  n_iterations  \
 0            1.7       0.0001             0            37   
 
                          best_params  \
 0  {'alpha': 0.0001, 'epsilon': 1.7}   
 
                                           cv_results  
 0  {'mean_fit_time': [0.01767449378967285, 0.0114...  ,
 'model_params': {'alpha': 0.0001,
  'epsilon': 1.7,
  'fit_intercept': True,
  'max_iter': 2000,
  'tol': 1e-05,
  'warm_start': False},
 'feature_names': ['lag_1', 'lag_2', 'lag_3'

: 